# Team Harvey Wiley Cost Expiriment Notebook
In this notebook, we explore the relationship between costs, budgets, \& consumption.

**Usage Note - Updating Notebook**:
- If you make changes locally to your version of the notebook, you will need to manually update the notebook via the following steps
    1. Uncomment the code in the cell below and run it.
    2. Restart the notebook's kernel.
    3. Refresh the page. <br>
    **Warning** - Make sure that the aforementioned code is commented once you successfully updated the notebook.

In [20]:
### Uncomment this code to manually update the notebook
# !git reset --hard origin/main

In [21]:
import pandas as pd
import numpy as np
import cfe.regression as rgsn
import matplotlib.pyplot as plt

### Load in Data

In [23]:
%run nutrient_system.ipynb

### Price Change Policy Experiment

In [26]:
def plot_price_change_experiment(desired_food, desired_percentile, r, p):
    pbar, xhat, xbar, xref, qhat, fct0, c0 = get_pbqs(p, r, desired_percentile)
    x0 = xbar.quantile(desired_percentile / 100)
    
    P = np.geomspace(.01,10,50) * pbar[desired_food]

    ### Utility of household at desired_percentile, given prices
    u0 = r.indirect_utility(x0,pbar)

    plt.plot([r.demands(x0, change_price(p0 = p0, p = pbar, j = desired_food))[desired_food] for p0 in P], P)
    plt.plot([r.demands(u0, change_price(p0 = p0, p = pbar, j = desired_food),type = "Hicksian")[desired_food] for p0 in P], P)
    plt.ylabel('Price')
    plt.xlabel(desired_food)
    plt.legend(("Marshallian","Hicksian"))
    plt.show();

### Welfare Metrics

In [28]:
def compensating_variation(u0, p0, p1):
    x0 = r.expenditure(u0, p0)
    x1 = r.expenditure(u0, p1)
    return x1 - x0

def revenue(u0, p0, p1, type = 'Marshallian'):  
    dp = p1 - p0 # Change in prices
    c = r.demands(u0, p1, type = type)
    dp, c = dp.align(c, join = 'inner')
    return dp.T @ c


def deadweight_loss(u0, p0, p1):
    cv = compensating_variation(u0, p0, p1)
    return cv - revenue(u0,p0,p1,type='Hicksian')

### Price Changes, Revenue, and Compensating Variation Experiment

In [57]:
def plot_compensating_variation_experiment(desired_food, r, p, desired_percentile):
    pbar, xhat, xbar, xref, qhat, fct0, c0 = get_pbqs(p, r, desired_percentile)
    x0 = xbar.quantile(desired_percentile / 100)
    
    P = np.geomspace(.01,10,50) * pbar[desired_food]
    u0 = r.indirect_utility(x0,pbar)
    
    fig, ax1 = plt.subplots()

    ax1.plot(P,[compensating_variation(u0, pbar, change_price(p0 = p0, p = pbar, j = desired_food)) for p0 in P])
    ax1.set_xlabel(f"Price of {desired_food}")
    ax1.set_ylabel("Compensating Variation")

    ax1.plot(P,[revenue(u0,pbar,change_price(p0 = p0, p = pbar, j = desired_food),type='Hicksian') for p0 in P],'k')
    ax1.legend(('Compensating Variation','Revenue'))
    ax1.axhline(0, color = 'red')
    ax1.axvline(pbar.loc[desired_food], color = 'red')
    plt.show();

### Deadweight Loss

In [66]:
def plot_dwl_experiment(desired_food, r, p, desired_percentile):
    pbar, xhat, xbar, xref, qhat, fct0, c0 = get_pbqs(p, r, desired_percentile)
    x0 = xbar.quantile(desired_percentile / 100)
    
    P = np.geomspace(.01,10,50) * pbar[desired_food]
    u0 = r.indirect_utility(x0,pbar)

    fig, ax1 = plt.subplots()

    ax1.plot(P,[deadweight_loss(u0, pbar, change_price(p0 = p0, p = pbar, j = desired_food)) for p0 in P])
    ax1.set_xlabel("Price of %s" % desired_food)
    ax1.set_ylabel("Deadweight Loss")
    plt.show();

### Interactivity Development

In [34]:
def ppc_exp_interactive(r, p, food_choices):
    '''
    Description
    --------------------------------------------------
    This is a function that makes 
    plot_price_change_experiment() interactive.
    '''
    widget = interactive(plot_price_change_experiment,
                         r = widgets.fixed(r),
                         p = widgets.fixed(p),
                         desired_food = widgets.Dropdown(options = food_choices, 
                                                     description = "Desired Food"),
                                                 desired_percentile = widgets.IntSlider(value = 50,
                                                             min = 1,
                                                             max = 99,
                                                             step = 1,
                                                             description = 'Percentile:',
                                                             continuous_update = True
                                                            )
                        )
    display(widget)

In [53]:
def pcr_exp_interactive(r, p, food_choices):
    '''
    Description
    --------------------------------------------------
    This is a function that makes 
    plot_compensating_variation_experiment() 
    interactive.
    '''
    widget = interactive(plot_compensating_variation_experiment,
                         r = widgets.fixed(r),
                         p = widgets.fixed(p),
                         desired_food = widgets.Dropdown(options = food_choices, 
                                                     description = "Desired Food"),
                                                 desired_percentile = widgets.IntSlider(value = 50,
                                                             min = 1,
                                                             max = 99,
                                                             step = 1,
                                                             description = 'Percentile:',
                                                             continuous_update = True
                                                            )
                        )
    display(widget)

In [ ]:
def pdwl_exp_interactive(r, p, food_choices):
    '''
    Description
    --------------------------------------------------
    This is a function that makes 
    plot_dwl_experiment() interactive.
    '''
    widget = interactive(plot_dwl_experiment,
                         r = widgets.fixed(r),
                         p = widgets.fixed(p),
                         desired_food = widgets.Dropdown(options = food_choices, 
                                                     description = "Desired Food"),
                                                 desired_percentile = widgets.IntSlider(value = 50,
                                                             min = 1,
                                                             max = 99,
                                                             step = 1,
                                                             description = 'Percentile:',
                                                             continuous_update = True
                                                            )
                        )
    display(widget)